In [2]:
import mbuild as mb
from mbuild.lib.recipes import TiledCompound
import parmed as pmd
import numpy as np
import openmm
import warnings
from scipy.spatial.distance import pdist, squareform
import random
from copy import deepcopy
import openbabel as ob
warnings.simplefilter("ignore")

In [11]:
# Randomly select N sites from `coords`, with a minimum of X distance between them
# Usage: select_coordinates(coords, N, X)
# clearly there are issues with this implementation, sometimes it can't find enough sites even when
# it should be possible

def select_coordinates(coords, num_to_select, min_distance):
    num_coords = len(coords)
    if num_to_select > num_coords:
        print("Warning: Not enough coordinates available to select.")
        return None

    distance_matrix = squareform(pdist(coords))

    valid_indices = set(range(num_coords))
    selected_indices = []

    for _ in range(num_to_select):
        if not valid_indices:
            print("Warning: Unable to find enough coordinates with the given constraints.")
            break

        index = random.choice(list(valid_indices))
        selected_indices.append(index)
        valid_indices.remove(index)

        too_close_indices = set(np.where(distance_matrix[index] < min_distance)[0])
        valid_indices -= too_close_indices

    return selected_indices

In [77]:
test_slab = mb.load("ZnS.14sqnm.pdb")
for S in test_slab:
    if S.element.symbol == 'S' and round(S.pos[2],3) == 0.681:
        # print(S)
        port = mb.Port(anchor=S, orientation=[0,0,1], separation=0.05)
        test_slab.add(port)

In [22]:
# # This is where you should visualize the system if something is going wrong
# test_slab.visualize(show_ports=True, backend="nglview")

In [78]:
# Get the coordinates of all the available bonding sites (surface Sulfur atoms)
coords = [port.pos for port in test_slab.all_ports()]

# Randomly select N sites from `coords`, with a minimum of X distance between them
# select_coordinates(coords, N, X)
# will throw a warning if unable to find specified N sites given coords + X criteria
selected_sites = select_coordinates(coords, 28, 0.75)
ports = [test_slab.all_ports()[site] for site in selected_sites]

In [79]:
len(ports)

28

In [80]:
# At each of the selected sites, add an oleic acid ligand

for site in ports:
    #should be coo- 
    oleic = mb.load('CCCCCCCCC=CCCCCCCCC(=O)O', smiles=True)
    oleic.add(mb.Port(anchor=oleic[18], separation=0.12, orientation=(oleic[18].pos - oleic[17].pos)))
    mb.force_overlap(move_this=oleic,
                     from_positions=oleic['Port[0]'],
                     to_positions=site)
    
    # oleic.remove_bond((site.anchor, oleic[18]))
    # slab.remove_bond((site.anchor, oleic[18]))
    test_slab.add(oleic, label="OLC[$]")

# for some reason I left this in here?? Need to check and remove
test_slab.rotate(np.pi, [1,1,0])
test_slab.translate([0,0,4])

In [81]:
## Visualize here to confirm that the distribution of ports is reasonable
test_slab.visualize(backend="nglview")

NGLWidget()

In [ ]:
## convert the single slab to parmed for proper file writing (mbuild mol2 writer doesn't work)
## write the slab file to PDB
## create the needed openbabel objects to convert file to BGF


# pmd_system = test_slab.to_parmed()
# pmd_system.write_pdb("new_single_slab_28lig.pdb")

# ob_conversion = ob.OBConversion()
# ob_conversion.SetInAndOutFormats("pdb", "bgf")
# molecule = ob.OBMol()

# ob_conversion.ReadFile(molecule, "new_single_slab_28lig.pdb")
# ob_conversion.WriteFile(molecule, "new_single_slab_28lig.bgf")

In [ ]:
# copy the original slab
# copy and flip to make an opposing slab
cool_slab = deepcopy(test_slab)
counter_slab = deepcopy(test_slab)
counter_slab.rotate(np.pi, [1,1,0])
counter_slab.translate([0,0,3])

# use the copied slabs to create a 2-slab system
system = mb.Compound()
system.add(cool_slab)
system.add(counter_slab)

In [ ]:
# visualize the system
system.visualize(backend="nglview")

In [ ]:
## As before, convert to parmed, save the file, and convert format to bgf with openbabel

# pmd_system = system.to_parmed()
# pmd_system.write_pdb("new_doubleslab_revised.pdb")

# ob_conversion = ob.OBConversion()
# ob_conversion.SetInAndOutFormats("pdb", "bgf")
# molecule = ob.OBMol()

# ob_conversion.ReadFile(molecule, "new_doubleslab_revised.pdb")
# ob_conversion.WriteFile(molecule, "new_doubleslab_revised.bgf")

## Unfortunately the system is too big to use with antechamber... 

Either need to assemble a smaller system here, ignore the missing torsion terms, manually set some stand-in torsion terms, get the torsion terms from a paper, or get the parameters from CHARMM-GUI

In [88]:
small_slab = mb.load("ZnS_2x2.pdb")
for S in small_slab:
    if S.element.symbol == 'S' and round(S.pos[2],3) == 0.681:
        # print(S)
        port = mb.Port(anchor=S, orientation=[0,0,1], separation=0.05)
        small_slab.add(port)

In [89]:
# Get the coordinates of all the available bonding sites (surface Sulfur atoms)
coords = [port.pos for port in small_slab.all_ports()]

# Randomly select N sites from `coords`, with a minimum of X distance between them
# select_coordinates(coords, N, X)
# will throw a warning if unable to find specified N sites given coords + X criteria
selected_sites = select_coordinates(coords, 4, 0.75)
ports = [small_slab.all_ports()[site] for site in selected_sites]

In [90]:
# At each of the selected sites, add an oleic acid ligand
for site in ports:
    #should be coo- 
    oleic = mb.load('CCCCCCCCC=CCCCCCCCC(=O)O', smiles=True)
    oleic.add(mb.Port(anchor=oleic[18], separation=0.12, orientation=(oleic[18].pos - oleic[17].pos)))
    mb.force_overlap(move_this=oleic,
                     from_positions=oleic['Port[0]'],
                     to_positions=site)
    
    # oleic.remove_bond((site.anchor, oleic[18]))
    # slab.remove_bond((site.anchor, oleic[18]))
    small_slab.add(oleic, label="OLC[$]")

In [91]:
small_slab.visualize(backend="nglview")

NGLWidget()

### Getting ready to incorporate the crosslinkable ligands

In [50]:
xlig = mb.load("../monte-carlo-test/struct_file/ns-bp_crosslinkable_ligand.mol2")
xlig.add(mb.Port(anchor=xlig[25], separation=0.25, orientation=(xlig[25].pos - xlig[14].pos)))
xlig.visualize(show_ports=True, backend="nglview")

NGLWidget()

## Trying to get periodicity to actually work

In [ ]:
mini_slab = mb.load("ZnS.mol2")

In [ ]:
mini_slab.visualize()

In [ ]:
mini_slab

In [ ]:
for i in mini_slab:
    print(i.pos)

In [80]:
unit_cell = mb.Box([0.545027, 0.545027, 0.545027], [90, 90, 90])

In [81]:
mini_slab.box = unit_cell

In [82]:
mini_slab

<Compound 46 particles, System box: Box: Lx=0.545027, Ly=0.545027, Lz=0.545027, xy=0.000000, xz=0.000000, yz=0.000000, , 56 bonds, id: 140311922202704>

In [83]:
mini_slab.periodicity=(True,True,True)

In [85]:
mini_tiles

<Compound2-1-1 92 particles, System box: Box: Lx=1.090054, Ly=0.545027, Lz=0.545027, xy=0.000000, xz=0.000000, yz=0.000000, , 112 bonds, id: 140311789801536>

In [65]:
mini_tiles[0].pos

array([0., 0., 0.])

In [75]:
pos1 = [round(p,2) for p in mini_tiles[0].pos]
pos1

[0.0, 0.0, 0.0]

In [93]:
for match in matches:
    mini_tiles.remove(match[1])

In [94]:
mini_tiles

<Compound2-1-1 71 particles, System box: Box: Lx=1.090054, Ly=0.545027, Lz=0.545027, xy=0.000000, xz=0.000000, yz=0.000000, , 84 bonds, id: 140311789801536>

In [ ]:
for port in mini_tiles.all_ports()

In [78]:
mini_tiles.save("min_test.xyz")

In [79]:
mini_tiles.energy_minimize()

*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested types.
*** Open Babel Warning  in Translate
  Cannot perform atom type translation: table cannot find requested